## Dataset List: 

1. PhysioNet 2021 Challenge

```
    Data Sources: 
    
    CPSC Database and CPSC-Extra Database
    INCART Database
    PTB and PTB-XL Database
    The Georgia 12-lead ECG Challenge (G12EC) Database
    Augmented Undisclosed Database
    Chapman-Shaoxing and Ningbo Database
    The University of Michigan (UMich) Database
```

## PhysioNet 2021 Challenge

The training data contains twelve-lead ECGs. The validation and test data contains twelve-lead, six-lead, four-lead, three-lead, and two-lead ECGs:

1. Twelve leads: I, II, III, aVR, aVL, aVF, V1, V2, V3, V4, V5, V6
2. Six leads: I, II, III, aVR, aVL, aVF
3. Four leads: I, II, III, V2
4. Three leads: I, II, V2
5. Two leads: I, II

Each ECG recording has one or more labels that describe cardiac abnormalities (and/or a normal sinus rhythm).

The Challenge data include annotated twelve-lead ECG recordings from six sources in four countries across three continents. These databases include over 100,000 twelve-lead ECG recordings with over 88,000 ECGs shared publicly as training data.

## Data Exploration & Examples

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# helper_functions.py
import helper_functions as hf